In [1]:
import pandas as pd
import numpy as np
import sys
import textblob
from tqdm import tqdm,tqdm_pandas
from joblib import Parallel, delayed
# from googletrans import Translator
from google_trans_new import google_translator

def read(x):
    return pd.read_csv(x)

def translate(filename, x):
    try:
#         return textblob.TextBlob(x).translate(from_lang="ru", to="en")
#         translator = Translator()
#         translation = translator.translate(x, src='ru', dest='en')        
        translator = google_translator()        
        translation = translator.translate(x, lang_src='ru', lang_tgt='en')
        file = open(filename, "a")
        file.write(translation)
        file.close()
        return translation
    except Exception as e:
        print(e)
        return e.__class__

In [19]:
# translating the Heading (quickly and ugly mode instead of reusing what was done for translating posts' content)
translation = []
translator = google_translator()
for idThread in df['IdThread'].unique():
    heading = df[df['IdThread'] == idThread]['Heading'].reset_index(drop=True)[0]
    heading_en = translator.translate(heading, lang_src='ru', lang_tgt='en')
    translation.append({'IdThread': idThread, 'Heading_en': heading_en})
headings = pd.DataFrame(translation).set_index('IdThread')
headings.head()
for idThread in headings.index:
    df.loc[df['IdThread'] == idThread, 'Heading_en'] = headings.loc[idThread]['Heading_en']

In [2]:
df = read("posts.csv")
df['IdPost'] = df['IdPost'].astype(int)
df = df.set_index('IdPost')
df.head()

,Content,Author,Likes,CitedPost,Timestamp,IdThread,Heading,Author-2,Forum
IdPost,,,,,,,,,
96068,Topic : phpBB 2.0.19 XSS\n\nSecurityAlert Id :...,18692,0,{},2006-01-05 20:50:00-02,12520,Новая XSS на форуме phpBB 2.0.19,18692,16
96083,***CITING***[about:blank]***CITING***Там Я гов...,21225,0,{},2006-01-05 21:31:00-02,12520,Новая XSS на форуме phpBB 2.0.19,18692,16
96091,"В админке phpbb есть пункт ""Разрешить HTML"". О...",20236,0,{},2006-01-05 22:07:00-02,12520,Новая XSS на форуме phpBB 2.0.19,18692,16
143919,"Незнаю, может кто-то такое и выкладывал... По ...",18829,0,{},2006-04-26 23:29:00-03,18262,Html скрипт загрузки вируса на удалёный комп!,18829,37
143920,Thx пробну***IMG***[styles/default/xenforo/cle...,21923,0,{},2006-04-26 23:38:00-03,18262,Html скрипт загрузки вируса на удалёный комп!,18829,37


In [3]:
len(df['IdThread'].unique())

219

# Translating posts following the strategy of concatening posts to reduce the number of requests and respecting the max number of characters per request allowed by Google Translator API.

In [4]:
df['PostTag'] = df.index.map(lambda idPost: '[IdPost:%d]' % idPost)

In [5]:
df['ContentLength'] = df['Content'].map(lambda x: len(x))
df['ContentLength'] = df['ContentLength'] + df['PostTag'].map(lambda x: len(x))
df['ContentLength'].describe()

count    47770.000000
mean       464.324869
std       1040.900802
min         17.000000
25%        121.000000
50%        211.000000
75%        418.000000
max      36412.000000
Name: ContentLength, dtype: float64

In [6]:
import re
def getIdsFromPost(post):
    ids = re.findall(r'\[IdPost:\s*(\d+)\s*\]', post)
    validIds = set(df.index) # it seems there are ids that fit the regex even though they are not from the used ids list
    ids = [id for id in ids if int(id) in validIds]
    return ids
# text = postsTranslated[0]
# ids = getIdsFromPost(text)
fileText = open("translated.txt", "r").read()
ids = getIdsFromPost(fileText)
print(len(fileText))
print(len(ids))

20777823
46931


In [7]:
def getPostsFromBatch(batch):
    ids = getIdsFromPost(batch)
    print(len(ids))
    posts = []
    begin = 0
    for idPost in ids:
        idPost = int(idPost)
        tagBegin = re.search(fr"\[IdPost:\s*{idPost}\s*\]", batch).start()
        tagEnd = re.search(fr"\[IdPost:\s*{idPost}\s*\]", batch).end()
        numTagCharacters = tagEnd - tagBegin
        end = tagBegin + numTagCharacters - 1
        post = batch[begin:end]
        posts.append({'IdPost': idPost, 'Content': post})
        begin = tagEnd + 1
        print("missing: %d " % (len(batch) - tagEnd))
#         batch = batch[begin:]
    return posts

In [26]:
# reconstructing posts, but now translated
translatedDf = pd.DataFrame(columns=['IdPost', 'Content'])
batch = open("translated.txt", "r").read()
posts = getPostsFromBatch(batch)
translatedDf = translatedDf.append(posts)
translatedDf = translatedDf.set_index('IdPost')

46931
missing: 20773608 
missing: 20772714 
missing: 20771837 
missing: 20770275 
missing: 20770180 
missing: 20769809 
missing: 20769491 
missing: 20769369 
missing: 20769194 
missing: 20768215 
missing: 20768116 
missing: 20767791 
missing: 20767723 
missing: 20767655 
missing: 20767480 
missing: 20767352 
missing: 20767323 
missing: 20765756 
missing: 20760952 
missing: 20760864 
missing: 20760682 
missing: 20760319 
missing: 20758190 
missing: 20758114 
missing: 20758075 
missing: 20758034 
missing: 20757728 
missing: 20757657 
missing: 20757406 
missing: 20757338 
missing: 20757132 
missing: 20756828 
missing: 20756693 
missing: 20755409 
missing: 20755114 
missing: 20754822 
missing: 20749943 
missing: 20749652 
missing: 20749354 
missing: 20748376 
missing: 20748291 
missing: 20747405 
missing: 20746672 
missing: 20746512 
missing: 20746237 
missing: 20746129 
missing: 20745919 
missing: 20744205 
missing: 20743036 
missing: 20742864 
missing: 20742462 
missing: 20742195 
missin

missing: 20445862 
missing: 20445835 
missing: 20445741 
missing: 20445548 
missing: 20445499 
missing: 20445473 
missing: 20445164 
missing: 20444924 
missing: 20444794 
missing: 20444653 
missing: 20444495 
missing: 20444052 
missing: 20443833 
missing: 20443695 
missing: 20443357 
missing: 20443236 
missing: 20442842 
missing: 20442520 
missing: 20442213 
missing: 20441744 
missing: 20441563 
missing: 20441505 
missing: 20441360 
missing: 20440875 
missing: 20440280 
missing: 20439948 
missing: 20439758 
missing: 20439660 
missing: 20439524 
missing: 20439496 
missing: 20439350 
missing: 20439164 
missing: 20438924 
missing: 20438870 
missing: 20438741 
missing: 20438399 
missing: 20438139 
missing: 20437925 
missing: 20437200 
missing: 20436910 
missing: 20436664 
missing: 20436193 
missing: 20435519 
missing: 20435302 
missing: 20431745 
missing: 20431652 
missing: 20431428 
missing: 20431313 
missing: 20431246 
missing: 20431171 
missing: 20430938 
missing: 20430871 
missing: 204

missing: 20268024 
missing: 20267840 
missing: 20267623 
missing: 20267425 
missing: 20266961 
missing: 20266880 
missing: 20266553 
missing: 20266334 
missing: 20266193 
missing: 20265914 
missing: 20265586 
missing: 20265458 
missing: 20265295 
missing: 20265135 
missing: 20265094 
missing: 20264993 
missing: 20264754 
missing: 20264513 
missing: 20264464 
missing: 20263909 
missing: 20263612 
missing: 20263460 
missing: 20263036 
missing: 20261649 
missing: 20261514 
missing: 20261343 
missing: 20260951 
missing: 20260894 
missing: 20260798 
missing: 20260565 
missing: 20260367 
missing: 20260105 
missing: 20259696 
missing: 20259172 
missing: 20259004 
missing: 20258954 
missing: 20258726 
missing: 20258555 
missing: 20258267 
missing: 20258172 
missing: 20257917 
missing: 20257667 
missing: 20257506 
missing: 20257395 
missing: 20257162 
missing: 20257034 
missing: 20256892 
missing: 20256557 
missing: 20256132 
missing: 20255515 
missing: 20255406 
missing: 20255219 
missing: 202

missing: 20087602 
missing: 20087293 
missing: 20087099 
missing: 20086660 
missing: 20086233 
missing: 20086149 
missing: 20085761 
missing: 20085348 
missing: 20085091 
missing: 20084514 
missing: 20084233 
missing: 20084132 
missing: 20084097 
missing: 20083843 
missing: 20083811 
missing: 20082121 
missing: 20081719 
missing: 20081568 
missing: 20079005 
missing: 20077893 
missing: 20076912 
missing: 20076859 
missing: 20074639 
missing: 20073076 
missing: 20072878 
missing: 20072788 
missing: 20072442 
missing: 20072376 
missing: 20071046 
missing: 20070975 
missing: 20070838 
missing: 20070714 
missing: 20070444 
missing: 20070105 
missing: 20068518 
missing: 20068368 
missing: 20067565 
missing: 20067402 
missing: 20066940 
missing: 20066231 
missing: 20066132 
missing: 20066085 
missing: 20065782 
missing: 20065519 
missing: 20065363 
missing: 20064535 
missing: 20064157 
missing: 20063638 
missing: 20063223 
missing: 20063158 
missing: 20063088 
missing: 20062915 
missing: 200

missing: 19897875 
missing: 19897512 
missing: 19897320 
missing: 19897277 
missing: 19896945 
missing: 19896795 
missing: 19896648 
missing: 19896604 
missing: 19896540 
missing: 19896175 
missing: 19896080 
missing: 19895811 
missing: 19895738 
missing: 19895591 
missing: 19895454 
missing: 19895400 
missing: 19895303 
missing: 19895183 
missing: 19895050 
missing: 19894923 
missing: 19893757 
missing: 19893262 
missing: 19893142 
missing: 19893082 
missing: 19892932 
missing: 19892835 
missing: 19892716 
missing: 19892611 
missing: 19892496 
missing: 19892249 
missing: 19892021 
missing: 19891949 
missing: 19891698 
missing: 19891268 
missing: 19891183 
missing: 19890701 
missing: 19890701 
missing: 19890550 
missing: 19890384 
missing: 19890230 
missing: 19889836 
missing: 19889738 
missing: 19889470 
missing: 19889052 
missing: 19888944 
missing: 19888865 
missing: 19888689 
missing: 19888544 
missing: 19888418 
missing: 19887982 
missing: 19887725 
missing: 19887627 
missing: 198

...

missing: 19280946 
missing: 19280781 
missing: 19280531 
missing: 19280446 
missing: 19280322 
missing: 19280058 
missing: 19279709 
missing: 19279414 
missing: 19279341 
missing: 19279170 
missing: 19278958 
missing: 19277886 
missing: 19277679 
missing: 19277377 
missing: 19277247 
missing: 19277168 
missing: 19277076 
missing: 19276807 
missing: 19276207 
missing: 19276139 
missing: 19275575 
missing: 19274834 
missing: 19274509 
missing: 19274387 
missing: 19274128 
missing: 19273801 
missing: 19273448 
missing: 19273337 
missing: 19272883 
missing: 19272627 
missing: 19272433 
missing: 19272033 
missing: 19271901 
missing: 19271625 
missing: 19271563 
missing: 19271405 
missing: 19270920 
missing: 19270777 
missing: 19270666 
missing: 19270305 
missing: 19270105 
missing: 19269643 
missing: 19269391 
missing: 19269244 
missing: 19267474 
missing: 19267338 
missing: 19267001 
missing: 19266688 
missing: 19266545 
missing: 19266441 
missing: 19266208 
missing: 19266041 
missing: 192

missing: 19145728 
missing: 19145542 
missing: 19145425 
missing: 19145312 
missing: 19144902 
missing: 19144120 
missing: 19144048 
missing: 19143903 
missing: 19143609 
missing: 19143029 
missing: 19142968 
missing: 19142770 
missing: 19142730 
missing: 19141605 
missing: 19141445 
missing: 19141367 
missing: 19141097 
missing: 19140728 
missing: 19140617 
missing: 19140381 
missing: 19140236 
missing: 19140113 
missing: 19139947 
missing: 19139817 
missing: 19139730 
missing: 19139370 
missing: 19139226 
missing: 19138562 
missing: 19138416 
missing: 19137298 
missing: 19136722 
missing: 19136375 
missing: 19136119 
missing: 19135771 
missing: 19135687 
missing: 19135604 
missing: 19135179 
missing: 19134442 
missing: 19133918 
missing: 19133885 
missing: 19133622 
missing: 19132719 
missing: 19131490 
missing: 19130849 
missing: 19130157 
missing: 19130087 
missing: 19129087 
missing: 19128938 
missing: 19128838 
missing: 19128324 
missing: 19127941 
missing: 19127862 
missing: 191

missing: 19006976 
missing: 19006904 
missing: 19006830 
missing: 19006497 
missing: 19005245 
missing: 19005179 
missing: 19004821 
missing: 19004746 
missing: 19004563 
missing: 19004315 
missing: 19004280 
missing: 19004140 
missing: 19004050 
missing: 19003733 
missing: 19003693 
missing: 19003339 
missing: 19003077 
missing: 19002661 
missing: 19002627 
missing: 19002523 
missing: 19002139 
missing: 19001767 
missing: 19001676 
missing: 19001598 
missing: 19001361 
missing: 19001282 
missing: 19001216 
missing: 19001131 
missing: 19000662 
missing: 19000579 
missing: 19000523 
missing: 19000397 
missing: 19000300 
missing: 18999870 
missing: 18998849 
missing: 18998463 
missing: 18998174 
missing: 18997663 
missing: 18997360 
missing: 18997178 
missing: 18997143 
missing: 18996870 
missing: 18996240 
missing: 18995945 
missing: 18995834 
missing: 18995766 
missing: 18995418 
missing: 18994979 
missing: 18994753 
missing: 18994431 
missing: 18994046 
missing: 18993720 
missing: 189

missing: 18847092 
missing: 18846898 
missing: 18846799 
missing: 18846639 
missing: 18846193 
missing: 18845690 
missing: 18845528 
missing: 18845199 
missing: 18845149 
missing: 18844998 
missing: 18844544 
missing: 18844273 
missing: 18843722 
missing: 18843624 
missing: 18843520 
missing: 18843227 
missing: 18843069 
missing: 18842943 
missing: 18842746 
missing: 18842053 
missing: 18841952 
missing: 18841464 
missing: 18841326 
missing: 18841190 
missing: 18840945 
missing: 18840810 
missing: 18840633 
missing: 18840434 
missing: 18840128 
missing: 18839919 
missing: 18839801 
missing: 18839534 
missing: 18838632 
missing: 18838509 
missing: 18838308 
missing: 18838122 
missing: 18837887 
missing: 18837712 
missing: 18837412 
missing: 18837089 
missing: 18836732 
missing: 18836671 
missing: 18836363 
missing: 18836212 
missing: 18835913 
missing: 18835852 
missing: 18835637 
missing: 18835499 
missing: 18835410 
missing: 18835348 
missing: 18835141 
missing: 18834763 
missing: 188

KeyboardInterrupt: 

In [38]:
df.shape

(47474, 11)

In [39]:
translatedDf.shape

(46931, 2)

In [41]:
df.loc[4269730]

Content          Разбирал и собирал apktool. Подписывал ключем,...
Author                                                      330336
Likes                                                            0
CitedPost                                                       {}
Timestamp                                   2018-12-17 07:18:00-02
IdThread                                                     48176
Heading                     Реверсинг. Задай вопрос - получи ответ
Author-2                                                     36960
Forum                                                           94
PostTag                                           [IdPost:4269730]
ContentLength                                                  416
Name: 4269730, dtype: object

In [44]:
translatedDf.loc[4269730]

Content    Disassembled and assembled apktool. I signed i...
Name: 4269730, dtype: object

In [45]:
translatedDf.to_csv("translated_posts.csv")

In [27]:
alreadyTranslatedIds = set(translatedDf.index)

In [28]:
# googleLimit = 15000 # google translator character limit per translation
googleLimit = 5000 # NEW google translator character limit per translation
batchesToTranslate = []
batch = ''

originalSize = df.shape[0]
df = df[df['ContentLength'] <= googleLimit]
print('Contents too large removed:%d' % (originalSize-df.shape[0]))

for idPost in df.index:
    if (idPost in alreadyTranslatedIds):
        continue
    postLength = df.loc[idPost, 'ContentLength']
    if (len(batch) + postLength <= googleLimit):
        batch += df.loc[idPost, 'Content'] + df.loc[idPost, 'PostTag']
    else:
        batchesToTranslate.append(batch)
        batch = df.loc[idPost, 'Content'] + df.loc[idPost, 'PostTag']

Contents too large removed:0


In [34]:
len(batchesToTranslate)

566

In [36]:
from functools import partial
# Translate concatenated posts
bound_translation = partial(translate, "translated.txt")
postsTranslated = list(map(bound_translation, batchesToTranslate))